# MLP for the bank dataset

In [113]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os

In [114]:
FILE_PATH = 'data/bank/bank_normalized.csv'
rawData = pd.read_csv(FILE_PATH)

In [115]:
rawData.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,oct,sep,fri,mon,thu,tue,wed,failure,nonexistent,success
0,0.481481,0.053070,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,0,1,0,0,0,0,1,0
1,0.493827,0.030297,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,0,1,0,0,0,0,1,0
2,0.246914,0.045954,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,0,1,0,0,0,0,1,0
3,0.283951,0.030704,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,0,1,0,0,0,0,1,0
4,0.481481,0.062424,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,0,1,0,0,0,0,1,0


In [116]:
rawData.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y',
       'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
       'retired', 'self-employed', 'services', 'student', 'technician',
       'unemployed', 'unknown', 'divorced', 'married', 'single', 'unknown.1',
       'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate',
       'professional.course', 'university.degree', 'unknown.2', 'no',
       'unknown.3', 'yes', 'no.1', 'unknown.4', 'yes.1', 'no.2', 'unknown.5',
       'yes.2', 'cellular', 'telephone', 'apr', 'aug', 'dec', 'jul', 'jun',
       'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon', 'thu', 'tue', 'wed',
       'failure', 'nonexistent', 'success'],
      dtype='object')

In [117]:
yLabel = 'y'
keys = [i for i in rawData.keys() if i != yLabel]
nInstances = rawData.shape[0]
nInput = rawData.shape[1] - 1
nClasses = rawData[yLabel].unique().shape[0]

print("nInstances = {}, nInput = {}, nClasses = {}".format(nInstances, nInput, nClasses))

nInstances = 41188, nInput = 63, nClasses = 2


In [118]:
testSize = 0.25

In [119]:
learningRate = 0.001
trainingEpochs = 1000
batchSize = 100
displayStep = 20
hiddenSize = 256
activationFuncOut = tf.nn.tanh
stdDev = 0.1
randomState = 2018

In [120]:
data = rawData[keys].get_values()
labels = rawData[yLabel].get_values()

In [121]:
data.shape

(41188, 63)

In [122]:
labels.dtype

dtype('float64')

Code the labels

In [123]:
labels_ = np.zeros((nInstances, nClasses))
labels_[np.arange(nInstances), labels.astype('int32')] = 1

In [124]:
dataTrain, dataTest, labelsTrain, labelsTest = train_test_split(data, labels_,
                                                               test_size = testSize,
                                                               random_state = randomState)

In [125]:
print("Shapes:\ndataTrain: {}, dataTest: {},\nlabelsTrain: {}, labelsTest: {}".format(dataTrain.shape,
                                                                                     dataTest.shape,
                                                                                     labelsTrain.shape,
                                                                                     labelsTest.shape))

Shapes:
dataTrain: (30891, 63), dataTest: (10297, 63),
labelsTrain: (30891, 2), labelsTest: (10297, 2)


In [126]:
net_nInput = nInput
net_nHidden1 = hiddenSize
net_nHidden2 = hiddenSize
net_nHidden3 = hiddenSize
net_nHidden4 = hiddenSize
net_nClasses = nClasses

tf.reset_default_graph()

In [127]:
X = tf.placeholder(dtype = tf.float64, shape = (None, nInput))
# Labels
y = tf.placeholder(dtype = tf.float64, shape = (None, nClasses))

Dropout: the probability of keeping a neuron active (otherwise the neuron is set to zero).

In [128]:
dropoutKeepProb = tf.placeholder(tf.float64)

In [129]:
def buildDeepMLPClassifier(_X, _weights, _biases, _dropoutKeepProb):
    layer1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), _dropoutKeepProb)
    layer2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), _dropoutKeepProb)
    layer3 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), _dropoutKeepProb)
    layer4 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), _dropoutKeepProb)
    out = activationFuncOut(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    return out

In [130]:
weights = {
    'h1': tf.Variable(tf.random_normal((net_nInput, net_nHidden1), stddev = stdDev, dtype = tf.float64)),
    'h2': tf.Variable(tf.random_normal((net_nHidden1, net_nHidden2), stddev = stdDev, dtype = tf.float64)),
    'h3': tf.Variable(tf.random_normal((net_nHidden2, net_nHidden3), stddev = stdDev, dtype = tf.float64)),
    'h4': tf.Variable(tf.random_normal((net_nHidden3, net_nHidden4), stddev = stdDev, dtype = tf.float64)),
    'out': tf.Variable(tf.random_normal((net_nHidden4, net_nClasses), stddev = stdDev, dtype = tf.float64))
}
biases = {
    'b1': tf.Variable(tf.random_normal((net_nHidden1,), dtype = tf.float64)),
    'b2': tf.Variable(tf.random_normal((net_nHidden2,), dtype = tf.float64)),
    'b3': tf.Variable(tf.random_normal((net_nHidden3,), dtype = tf.float64)),
    'b4': tf.Variable(tf.random_normal((net_nHidden4,), dtype = tf.float64)),
    'out': tf.Variable(tf.random_normal((net_nClasses,), dtype = tf.float64))
}

In [131]:
pred = buildDeepMLPClassifier(X, weights, biases, dropoutKeepProb)

In [132]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

In [133]:
optimizer = tf.train.AdamOptimizer(learning_rate = learningRate).minimize(cost)

Additional parameters for computing the classification accuracy:

In [134]:
correctPrediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float64))

## Training 

In [135]:
initOp = tf.global_variables_initializer()

In [136]:
sess = tf.Session()
sess.run(initOp)

In [137]:
for epoch in range(trainingEpochs):
    avgCost = 0.0
    totalBatches = int(dataTrain.shape[0] / batchSize)
    # Loop over all batches
    for i in range(totalBatches):
        randidx = np.random.randint(dataTrain.shape[0], size = batchSize)
        batchX = dataTrain[randidx, :]
        batchY = labelsTrain[randidx, :]
        sess.run(optimizer, feed_dict = {X: batchX, y: batchY, dropoutKeepProb: 0.9})
        avgCost += sess.run(cost, 
                            feed_dict = {X: batchX, y: batchY, dropoutKeepProb: 1.0}) / totalBatches
    if epoch % displayStep == 0:
        print("Epoch {}/{}: avg. cost {}".format(epoch, trainingEpochs, avgCost))
        trainAcc = sess.run(accuracy, feed_dict = {X: batchX, y: batchY, dropoutKeepProb: 1.0})
        print("Training accuracy: {}".format(trainAcc))

Epoch 0/1000: avg. cost 0.35491395240100215
Training accuracy: 0.9
Epoch 20/1000: avg. cost 0.3569929460687068
Training accuracy: 0.88
Epoch 40/1000: avg. cost 0.3516682707107589
Training accuracy: 0.9
Epoch 60/1000: avg. cost 0.3542656733809655
Training accuracy: 0.93
Epoch 80/1000: avg. cost 0.3506293097445214
Training accuracy: 0.88
Epoch 100/1000: avg. cost 0.3495254136405275
Training accuracy: 0.9
Epoch 120/1000: avg. cost 0.3524474915625439
Training accuracy: 0.82
Epoch 140/1000: avg. cost 0.3542007383157631
Training accuracy: 0.87
Epoch 160/1000: avg. cost 0.35575917987418354
Training accuracy: 0.87
Epoch 180/1000: avg. cost 0.3497202188352367
Training accuracy: 0.9
Epoch 200/1000: avg. cost 0.3529020370170362
Training accuracy: 0.9
Epoch 220/1000: avg. cost 0.351018920133919
Training accuracy: 0.93
Epoch 240/1000: avg. cost 0.35095398506897907
Training accuracy: 0.89
Epoch 260/1000: avg. cost 0.3587461928611624
Training accuracy: 0.92
Epoch 280/1000: avg. cost 0.351473465588451

In [139]:
testAcc = sess.run(accuracy, feed_dict = {X: dataTest, y: labelsTest, dropoutKeepProb: 1.0})
print("Prediction/classification accuracy: {}".format(testAcc))

Prediction/classification accuracy: 0.8887054481887928


In [140]:
sess.close()